# Tự động tính đạo hàm với torch.autograd

## Sơ đồ mạng Neural cho như sau
<img src="nn.png" alt="Mạng Neural y=w*x+b" width="600">

In [27]:
import torch
x = torch.ones(5)
y = torch.zeros(3)

w = torch.randn(5,3, requires_grad=True)

b = torch.randn(3, requires_grad=True)
z = torch.matmul(x,w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z,y)

In [24]:
print(f"Gradient function for z: {z.grad_fn}")
print(f"Gradient function for loss: {loss.grad_fn}")

Gradient function for z: <AddBackward0 object at 0x7770d99c91e0>
Gradient function for loss: <BinaryCrossEntropyWithLogitsBackward0 object at 0x7770c136cdf0>


In [28]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.0168, 0.0349, 0.2942],
        [0.0168, 0.0349, 0.2942],
        [0.0168, 0.0349, 0.2942],
        [0.0168, 0.0349, 0.2942],
        [0.0168, 0.0349, 0.2942]])
tensor([0.0168, 0.0349, 0.2942])


## Tắt tính Gradient Decent (TH muốn đóng băng parameter của mô hinh)

In [29]:
z = torch.matmul(x,w) + b
print(f"z.grad={z.requires_grad} \n")
with torch.no_grad():
    z = torch.matmul(x,w) + b
print(f"z.no_grad={z.requires_grad} \n")

z.grad=True 

z.no_grad=False 



## Tính toán trên đồ thị
### autograd_ tạo một bản ghi của dữ liệu (tensor) và tính toán trên một đồ thị có hướng không chu trình (DAG - directed acyclic graph )
### nó bao gồm một đối tượng Function.
### Trên đồ thị, các nút là input tensor, các nút gốc là output tensors
### Bằng cách theo dõi đồ thị từ nút gốc đến nút lá, ta có thể tính toán gradients sử dụng quy tắc dây chuyền (chain rule)

## Tensor Gradient và phép nhân Jacobian

In [30]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"First call\n{inp.grad}")
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])
